In [4]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #just in case, regularly update your lucem_illud with the following code: pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import pandas #gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import wordcloud #Makes word clouds
import numpy as np #For divergences/distances
import scipy #For divergences/distances
import seaborn as sns #makes our plots look nicer
import sklearn.manifold #For a manifold plot
import json #For API responses
import urllib.parse #For joining urls

# comp-linguistics
import spacy

#Displays the graphs
import graphviz #You also need to install the command line graphviz

#These are from the standard library
import os.path
import zipfile
import subprocess
import io
import tempfile

#This 'magic' command makes the plots work better
#in the notebook, don't use it outside of a notebook
%matplotlib inline

In [5]:
tv_corpus = "/Users/theletterfifteen/ComputationalContent/Homework-Notebooks-copy/week-2/Corpora/TV" 
# corpus_name = "TV"

In [6]:
def loadcorpus(corpus_name, corpus_style="text"):
    texts_raw = {}
    for file in os.listdir(corpus_name + "/"):
        if corpus_style in file:
            print(file)
            zfile = zipfile.ZipFile(corpus_name + "/" + file)
            for file in zfile.namelist():
                texts_raw[file] = []
                with zfile.open(file) as f:
                    for line in f:
                        texts_raw[file].append(line)
    return texts_raw

In [7]:
tv_raw = loadcorpus(tv_corpus)

text_13_idi.zip
text_16_qll.zip
text_32_ldf.zip
text_19_gvc.zip
text_09_jex.zip
text_05_nko.zip
text_17_arp.zip
text_01_ote.zip
text_28_rfy.zip
text_31_akv.zip
text_22_etp.zip
text_11_uoy.zip
text_14_lnc.zip
text_06_eyp.zip
text_08_loh.zip
text_33_kje.zip
text_30_wkp.zip
text_07_oma.zip
text_03_mnq.zip
text_21_fqa.zip
text_29_oye.zip
text_27_fle.zip
text_23_fmh.zip
text_12_rcq.zip
text_00_myn.zip
text_10_aoy.zip
text_04_mlq.zip
text_20_cde.zip
text_02_mqu.zip
text_26_ngj.zip
text_24_ywo.zip
text_18_jfj.zip
text_25_byg.zip
text_15_guo.zip


In [8]:
zfile = zipfile.ZipFile(tv_corpus + "/sources_tv.zip")
source= []

In [9]:
for file in zfile.namelist():
    with zfile.open(file) as f:
        for line in f:
            source.append(line)

In [10]:
tv_texts = {}

In [12]:
def clean_raw_text(raw_texts):
    clean_texts = []
    for text in raw_texts:
        try:
            text = text.decode("utf-8")
            clean_text = text.replace(" \'m", "'m").replace(" \'ll", "'ll").replace(" \'re", "'re").replace(" \'s", "'s").replace(" \'re", "'re").replace(" n\'t", "n't").replace(" \'ve", "'ve").replace(" /'d", "'d")
            clean_texts.append(clean_text)
        except AttributeError:
            # print("ERROR CLEANING")
            # print(text)
            continue
        except UnicodeDecodeError:
            # print("Unicode Error, Skip")
            continue
    return clean_texts

In [13]:
for files in tv_raw:
    shows = clean_raw_text(tv_raw[files][1:])
    for show in shows:
        txts = lucem_illud.word_tokenize(show)
        try:
            tv_texts[txts[0][2:]] = txts[1:]
        except IndexError:
            continue

In [14]:
tv_df = pandas.DataFrame(columns=["Series", "Episode", "Genre", "Year", "Country", "Tokenized Texts"])

In [15]:
for show in source[3:]:
    try:
        tid, fileid, total_words, genre, year, country, lang, seriesid, imdb, series, episode = show.decode("utf-8").split("\t")
    except UnicodeDecodeError:
        continue
    try:
        tv_df.loc[fileid.strip()] = [series.strip(), episode.strip(), genre.strip(), year.strip(), country.strip(), tv_texts[fileid.strip()]]
    except KeyError:
        continue

In [16]:
tv_df.head()

,Series,Episode,Genre,Year,Country,Tokenized Texts
6814479,A Moody Christmas,Decapod Crustaceans,Comedy,2012,Australia,"[Engineidles, Come, on, come, on, come, on, MA..."
6814480,A Moody Christmas,I'm Walt Roskow,Comedy,2012,Australia,"[KEVIN, The, whole, business, gone, Thanks, to..."
6814482,A Moody Christmas,Last Minute Airfare,Comedy,2012,Australia,"[Shivers, Remember, when, we, used, to, say, T..."
6814478,A Moody Christmas,Operation Sex Via the Homeless,Comedy,2012,Australia,"[Sighs, MAREE, Sean, we, 're, leaving, KEVIN, ..."
6814477,A Moody Christmas,Separate Seats,Comedy,2012,Australia,"[WOMAN, ON, LOUDSPEAKER, Final, call, for, Fli..."


In [64]:
dw_df = tv_df[tv_df["Series"].isin(["Doctor Who"])]
dw_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
6847708,Doctor Who,An Unearthly Child,"Adventure, Drama, Family",1963,UK,"[Doctor, Who, Episode1, Season, 1, Episode, 1,..."
4972879,Doctor Who,The Cave of Skulls,"Adventure, Drama, Family",1963,UK,"[DOCTORWHOTHEME, Where, is, the, fire, Za, mak..."
5562046,Doctor Who,The Dead Planet,"Adventure, Drama, Family",1963,UK,"[Well, I, suggest, before, we, go, outside, an..."
5348186,Doctor Who,The Firemaker,"Adventure, Drama, Family",1963,UK,"[Back, Get, back, They, are, coming, Za, and, ..."
5082927,Doctor Who,The Forest of Fear,"Adventure, Drama, Family",1963,UK,"[DOCTORWHOTHEME, Oh, I, 'm, sorry, I, 'm, sorr..."
...,...,...,...,...,...,...
7002007,Doctor Who,The Lie of the Land,"Adventure, Drama, Family",2017,UK,"[The, Monks, have, been, with, us, from, the, ..."
6949959,Doctor Who,The Pilot,"Adventure, Drama, Family",2017,UK,"[CLOCK, TICKS, RHYTHMICALLY, FOOTSTEPS, APPROA..."
6995511,Doctor Who,The Pyramid at the End of the World,"Adventure, Drama, Family",2017,UK,"[A, few, months, ago, the, Veritas, was, trans..."
6964420,Doctor Who,Thin Ice,"Adventure, Drama, Family",2017,UK,"[OK, I, have, questions, You, never, said, we,..."


In [75]:
dw_df.to_csv('Doctor_Who.csv', index=False) 

In [76]:
buffy_df = tv_df[tv_df["Series"].isin(["Buffy the Vampire Slayer"])]
buffy_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3090627,Buffy the Vampire Slayer,Unaired Pilot,"Action, Drama, Fantasy",1996,USA,"[The, chosen, one, She, alone, stands, against..."
4319643,Buffy the Vampire Slayer,Angel,"Action, Drama, Fantasy",1997,USA,"[Previously, on, Buffy, the, Vampire, Slayer, ..."
6272398,Buffy the Vampire Slayer,Halloween,"Action, Drama, Fantasy",1997,USA,"[I, know, Is, the, Bronze, so, not, happening,..."
4319642,Buffy the Vampire Slayer,"I, Robot... You, Jane","Action, Drama, Fantasy",1997,USA,"[In, every, generation, there, is, a, Chosen, ..."
5094402,Buffy the Vampire Slayer,Inca Mummy Girl,"Action, Drama, Fantasy",1997,USA,"[In, every, generation, there, is, a, Chosen, ..."
...,...,...,...,...,...,...
4615005,Buffy the Vampire Slayer,Potential,"Action, Drama, Fantasy",2003,USA,"[Previously, on, Buffy, the, Vampire, Slayer, ..."
4614966,Buffy the Vampire Slayer,Showtime,"Action, Drama, Fantasy",2003,USA,"[Previously, on, Buffy, the, Vampire, Slayer, ..."
4615008,Buffy the Vampire Slayer,Storyteller,"Action, Drama, Fantasy",2003,USA,"[Previously, on, Buffy, the, Vampire, Slayer, ..."
4615003,Buffy the Vampire Slayer,The Killer in Me,"Action, Drama, Fantasy",2003,USA,"[Previously, on, Buffy, the, Vampire, Slayer, ..."


In [77]:
buffy_df.to_csv('Buffy_the_Vampire_Slayer.csv', index=False) 

In [78]:
supernatural_df = tv_df[tv_df["Series"].isin(["Supernatural"])]
supernatural_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
4289570,Supernatural,Bloodlust,"Drama, Fantasy, Horror",2006,USA,"[I, think, he, wants, us, to, pick, up, what, ..."
4289571,Supernatural,Children Shouldn't Play with Dead Things,"Drama, Fantasy, Horror",2006,USA,"[Dad, wants, us, to, pick, up, where, he, left..."
5170233,Supernatural,Croatoan,"Drama, Fantasy, Horror",2006,USA,"[Dad, wants, us, to, pick, up, where, he, left..."
4906181,Supernatural,Crossroad Blues,"Drama, Fantasy, Horror",2006,USA,"[Dad, wants, us, to, pick, up, where, he, left..."
4289573,Supernatural,No Exit,"Drama, Fantasy, Horror",2006,USA,"[I, checked, the, fuses, They, 're, fine, It, ..."
...,...,...,...,...,...,...
4171071,Supernatural,Mommy Dearest,"Drama, Fantasy, Horror",2011,"USA, Canada","[Bring, me, creatures, Aim, high, on, the, foo..."
4163415,Supernatural,My Heart Will Go On,"Drama, Fantasy, Horror",2011,"USA, Canada","[Tires_screech, Yeah, I, think, these, are, Jo..."
4114471,Supernatural,The French Mistake,"Drama, Fantasy, Horror",2011,"USA, Canada","[SAM, Angels, got, to, have, something, to, do..."
4173977,Supernatural,The Man Who Would Be King,"Drama, Fantasy, Horror",2011,"USA, Canada","[We, 're, gon, na, find, a, way, to, beat, the..."


In [79]:
supernatural_df.to_csv('Supernatural.csv', index=False) 

In [80]:
startrek1_df = tv_df[tv_df["Series"].isin(["Star Trek"])]
startrek1_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
4279011,Star Trek,Balance of Terror,"Action, Adventure, Sci-Fi",1966,USA,"[Interesting, An, Arkturian, Macbeth, Watch, h..."
4279006,Star Trek,Dagger of the Mind,"Action, Adventure, Sci-Fi",1966,USA,"[Earth, style, distress, signal, SOS, I, 've, ..."
4279008,Star Trek,Miri,"Action, Adventure, Sci-Fi",1966,USA,"[All, sections, security, check, in, progress,..."
4279007,Star Trek,Mudd's Women,"Action, Adventure, Sci-Fi",1966,USA,"[That, should, make, a, good, specimen, Temper..."
4279015,Star Trek,Shore Leave,"Action, Adventure, Sci-Fi",1966,USA,"[The, ceremony, will, be, carried, on, all, vi..."
...,...,...,...,...,...,...
3114785,Star Trek,The Eye of the Beholder,"Animation, Action, Adventure",1974,USA,"[Space, the, final, frontier, These, are, the,..."
3114858,Star Trek,The Jihad,"Animation, Action, Adventure",1974,USA,"[Space, the, final, frontier, These, are, the,..."
3116649,Star Trek,The Pirates of Orion,"Animation, Action, Adventure",1974,USA,"[Space, the, final, frontier, These, are, the,..."
3116939,Star Trek,The Practical Joker,"Animation, Action, Adventure",1974,USA,"[Space, the, final, frontier, These, are, the,..."


In [81]:
startrek1_df.to_csv('Star_Trek.csv', index=False) 

In [82]:
startrek2_df = tv_df[tv_df["Series"].isin(["Star Trek: The Next Generation"])]
startrek2_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
4817168,Star Trek: The Next Generation,Code of Honor,"Action, Adventure, Mystery",1987,USA,"[PICARD, Captain, 's, log, stardate, 41235.25,..."
4990272,Star Trek: The Next Generation,Encounter at Farpoint,"Action, Adventure, Mystery",1987,USA,"[Space, the, final, frontier, These, are, the,..."
5174063,Star Trek: The Next Generation,Hide and Q,"Action, Adventure, Mystery",1987,USA,"[Captain, 's, log, stardate, 41590.5, Having, ..."
3181507,Star Trek: The Next Generation,Justice,"Action, Adventure, Mystery",1987,USA,"[Captain, 's, log, star, date, 41249.3, We, ha..."
3182064,Star Trek: The Next Generation,The Battle,"Action, Adventure, Mystery",1987,USA,"[Captain, 's, log, star, date, 41723.9, Follow..."
...,...,...,...,...,...,...
5148825,Star Trek: The Next Generation,Masks,"Action, Adventure, Mystery",1994,USA,"[That, 's, very, good, Well, that, 's, very, n..."
161407,Star Trek: The Next Generation,Preemptive Strike,"Action, Adventure, Mystery",1994,USA,"[Captain, 's, log, stardate, 47941.7, The, Ent..."
4907858,Star Trek: The Next Generation,Sub Rosa,"Action, Adventure, Mystery",1994,USA,"[Most, people, will, remember, my, grandmother..."
161392,Star Trek: The Next Generation,The Pegasus,"Action, Adventure, Mystery",1994,USA,"[I, do, n't, see, why, we, have, to, do, this,..."


In [83]:
startrek2_df.to_csv('Star_Trek_The_Next_Generation.csv', index=False) 

In [84]:
startrek3_df = tv_df[tv_df["Series"].isin(["Star Trek: Deep Space Nine"])]
startrek3_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
4049509,Star Trek: Deep Space Nine,A Man Alone,"Action, Adventure, Drama",1993,USA,"[Yes, Julian, How, did, you, know, it, was, me..."
4049513,Star Trek: Deep Space Nine,Babel,"Action, Adventure, Drama",1993,USA,"[Look, save, your, energy, We, 'll, have, you,..."
5102712,Star Trek: Deep Space Nine,Battle Lines,"Action, Adventure, Drama",1993,USA,"[It, 's, funny, that, the, files, were, in, Op..."
4049514,Star Trek: Deep Space Nine,Captive Pursuit,"Action, Adventure, Drama",1993,USA,"[I, 'm, not, what, a, lot, of, people, think, ..."
4049559,Star Trek: Deep Space Nine,Cardassians,"Action, Adventure, Drama",1993,USA,"[Tarkalean, tea, again, Doctor, Yes, And, you,..."
...,...,...,...,...,...,...
4201083,Star Trek: Deep Space Nine,The Changing Face of Evil,"Action, Adventure, Drama",1999,USA,"[Welcome, I, never, doubted, you, 'd, be, back..."
4201087,Star Trek: Deep Space Nine,The Dogs of War,"Action, Adventure, Drama",1999,USA,"[Anything, yet, She, 's, running, a, little, l..."
4201072,Star Trek: Deep Space Nine,The Emperor's New Cloak,"Action, Adventure, Drama",1999,USA,"[Have, you, ever, seen, anything, so, disgusti..."
4201088,Star Trek: Deep Space Nine,What You Leave Behind,"Action, Adventure, Drama",1999,USA,"[The, time, is, 0500, Yeah, all, right, Julian..."


In [85]:
startrek3_df.to_csv('Star_Trek_Deep_Space_Nine.csv', index=False) 

In [86]:
startrek4_df = tv_df[tv_df["Series"].isin(["Star Trek: Voyager"])]
startrek4_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3600748,Star Trek: Voyager,Caretaker,"Action, Adventure, Sci-Fi",1995,USA,"[Damage, report, Shields, at, 60, A, fuel, lin..."
3600761,Star Trek: Voyager,Cathexis,"Action, Adventure, Sci-Fi",1995,USA,"[Captain, 's, log, stardate, 48734.2, Sometime..."
4389908,Star Trek: Voyager,Cold Fire,"Action, Adventure, Sci-Fi",1995,USA,"[Ten, months, ago, the, crew, of, Voyager, was..."
4389912,Star Trek: Voyager,Elogium,"Action, Adventure, Sci-Fi",1995,USA,"[Oh, sir, we, um, we, 're, just, Yes, I, can, ..."
3600757,Star Trek: Voyager,Emanations,"Action, Adventure, Sci-Fi",1995,USA,"[Captain, 's, log, stardate, 48623.5, There, a..."
...,...,...,...,...,...,...
4201130,Star Trek: Voyager,Repentance,"Action, Adventure, Sci-Fi",2001,USA,"[This, is, Captain, Kathryn, Janeway, of, the,..."
4201128,Star Trek: Voyager,Shattered,"Action, Adventure, Sci-Fi",2001,USA,"[Hmm, Hmm, There, Excellent, You, recognized, ..."
4201132,Star Trek: Voyager,The Void,"Action, Adventure, Sci-Fi",2001,USA,"[For, our, third, course, Another, course, Luh..."
4201133,Star Trek: Voyager,Workforce: Part 1,"Action, Adventure, Sci-Fi",2001,USA,"[Excuse, me, can, you, tell, me, where, l, can..."


In [87]:
startrek4_df.to_csv('Star_Trek_Voyager.csv', index=False) 

In [88]:
startrek5_df = tv_df[tv_df["Series"].isin(["Star Trek: Enterprise"])]
startrek5_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3175250,Star Trek: Enterprise,Breaking the Ice,"Action, Adventure, Drama",2001,USA,"[Looks, like, we, 're, going, about, warp, 4.5..."
4879763,Star Trek: Enterprise,Broken Bow: Part 2,"Action, Adventure, Drama",2001,USA,"[You, 're, looking, for, Klaang, Why, Who, the..."
3175181,Star Trek: Enterprise,Broken Bow: Part I,"Action, Adventure, Drama",2001,USA,"[Where, no, man, has, gone, before, Dr, Cochra..."
3175254,Star Trek: Enterprise,Civilization,"Action, Adventure, Drama",2001,USA,"[Morning, everyone, Morning, sir, What, have, ..."
4818476,Star Trek: Enterprise,Cold Front,"Action, Adventure, Drama",2001,USA,"[This, was, n't, part, of, our, agreement, You..."
...,...,...,...,...,...,...
161295,Star Trek: Enterprise,Observer Effect,"Action, Adventure, Drama",2005,USA,"[You, 'll, win, in, eight, moves, You, 're, re..."
4265468,Star Trek: Enterprise,Terra Prime,"Action, Adventure, Drama",2005,USA,"[previously, on, Enterprise, Today, we, have, ..."
4954559,Star Trek: Enterprise,The Aenar,"Action, Adventure, Drama",2005,USA,"[Previously, on, Enterprise, From, what, I, 'v..."
5676295,Star Trek: Enterprise,These Are the Voyages...,"Action, Adventure, Drama",2005,USA,"[When, are, they, expecting, you, in, Brazil, ..."


In [89]:
startrek5_df.to_csv('Star_Trek_Enterprise.csv', index=False) 

In [90]:
agentsofshield_df = tv_df[tv_df["Series"].isin(["Agents of S.H.I.E.L.D."])]
agentsofshield_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
5203887,Agents of S.H.I.E.L.D.,0-8-4,"Action, Adventure, Drama",2013,USA,"[H.Q, S.H.I.E.L.D, 6, 1, 6, you, have, course,..."
5226034,Agents of S.H.I.E.L.D.,Eye Spy,"Action, Adventure, Drama",2013,USA,"[1x04, Eye, Spy, indistinct_chatter, music, Ma..."
5259272,Agents of S.H.I.E.L.D.,FZZT,"Action, Adventure, Drama",2013,USA,"[Thing, is, no, one, exactly, knows, what, hap..."
5236720,Agents of S.H.I.E.L.D.,Girl in the Flower Dress,"Action, Adventure, Drama",2013,USA,"[indistinct_talking, speaking_cantonese, blows..."
5193073,Agents of S.H.I.E.L.D.,Pilot,"Action, Adventure, Drama",2013,USA,"[Skye, The, secret, is, out, For, decades, you..."
...,...,...,...,...,...,...
6857088,Agents of S.H.I.E.L.D.,The Patriot,"Action, Adventure, Drama",2017,USA,"[Previously, on, Marvel, 's, Agents, of, S.H.I..."
6976561,Agents of S.H.I.E.L.D.,The Return,"Action, Adventure, Drama",2017,USA,"[Previously, on, Marvel, 's, Agents, of, S.H.I..."
6864035,Agents of S.H.I.E.L.D.,Wake Up,"Action, Adventure, Drama",2017,USA,"[Coulson, Previously, on, Marvel, 's, Agents, ..."
6939156,Agents of S.H.I.E.L.D.,What If...,"Action, Adventure, Drama",2017,USA,"[Previously, on, Marvel, 's, Agents, of, S.H.I..."


In [91]:
agentsofshield_df.to_csv('Agents_of_SHIELD.csv', index=False) 

In [92]:
sherlock_df = tv_df[tv_df["Series"].isin(["Sherlock"])]
sherlock_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3710151,Sherlock,A Study in Pink,"Crime, Drama, Mystery",2010,UK,"[43;288;TOOLONG, English, SRT, Subtitles, UF, ..."
3719519,Sherlock,The Blind Banker,"Crime, Drama, Mystery",2010,UK,"[41;1233;TOOLONG, English, SRT, Subtitles, UF,..."
3844491,Sherlock,The Great Game,"Crime, Drama, Mystery",2010,UK,"[Just, tell, me, what, happened, from, the, be..."
5141385,Sherlock,Unaired Pilot,"Crime, Drama, Mystery",2010,UK,"[How, 's, your, blog, going, Hmm, fine, Good, ..."
4459833,Sherlock,A Scandal in Belgravia,"Crime, Drama, Mystery",2012,UK,"[Who, are, you, Jim, Moriarty, Bye, Consulting..."
4944297,Sherlock,The Hounds of Baskerville,"Crime, Drama, Mystery",2012,UK,"[Hello, Are, you, all, right, What, is, it, de..."
5485884,Sherlock,The Reichenbach Fall,"Crime, Drama, Mystery",2012,UK,"[Why, today, Do, you, want, to, hear, me, say,..."
5459364,Sherlock,Many Happy Returns,"Crime, Drama, Mystery",2013,UK,"[HIMALAYAN_FLUTES_PLAY, LOW, MUTEDVOICE, You, ..."
5516543,Sherlock,His Last Vow,"Crime, Drama, Mystery",2014,UK,"[Mr, Magnussen, please, state, your, full, nam..."
5473049,Sherlock,The Empty Hearse,"Crime, Drama, Mystery",2014,UK,"[JOHN, Sherlock, Its, a, trick, Just, a, magic..."


In [93]:
sherlock_df.to_csv('Sherlock.csv', index=False) 

In [94]:
teenwolf_df = tv_df[tv_df["Series"].isin(["Teen Wolf"])]
teenwolf_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
4216842,Teen Wolf,Co-Captain,"Action, Comedy, Drama",2011,USA,"[Stiles, get, out, of, there, right, now, It, ..."
4223870,Teen Wolf,Code Breaker,"Action, Comedy, Drama",2011,USA,"[Previously, on, Teen, Wolf, Lydia, run, You, ..."
4220617,Teen Wolf,Formality,"Action, Comedy, Drama",2011,USA,"[Previously, on, Teen, Wolf, What, 's, in, her..."
4638603,Teen Wolf,Heart Monitor,"Action, Comedy, Drama",2011,USA,"[NARRATOR, Previously, on, Teen, Wolf, Oh, You..."
4209508,Teen Wolf,Lunatic,"Action, Comedy, Drama",2011,USA,"[Previously, on, Teen, Wolf, Hey, Lydia, You, ..."
...,...,...,...,...,...,...
6864092,Teen Wolf,Memory Found,"Action, Comedy, Drama",2017,USA,"[Previously, on, Teen, Wolf, Guess, who, knows..."
7073456,Teen Wolf,Pressure Test,"Action, Comedy, Drama",2017,USA,"[I, 'm, not, gon, na, tell, you, anything, I, ..."
7060122,Teen Wolf,Raw Talent,"Action, Comedy, Drama",2017,USA,"[SCOTT, Previously, on, Teen, Wolf, LIAM, Wher..."
6871525,Teen Wolf,Riders on the Storm,"Action, Comedy, Drama",2017,USA,"[Previously, on, Teen, Wolf, The, Ghost, Rider..."


In [95]:
teenwolf_df.to_csv('Teen_Wolf.csv', index=False) 

In [96]:
ouat_df = tv_df[tv_df["Series"].isin(["Once Upon a Time"])]
ouat_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
4261089,Once Upon a Time,Pilot,"Adventure, Fantasy, Romance",2011,USA,"[Made, by, Mikhel, for, subtitulos.es, =, =, S..."
4406682,Once Upon a Time,Snow Falls,"Adventure, Fantasy, Romance",2011,USA,"[Man, There, is, a, town, in, Maine, I, 'd, li..."
4419946,Once Upon a Time,That Still Small Voice,"Adventure, Fantasy, Romance",2011,USA,"[There, is, a, town, in, Maine, I, 'd, like, a..."
4430609,Once Upon a Time,The Heart is a Lonely Hunter,"Adventure, Fantasy, Romance",2011,USA,"[There, is, a, town, in, Maine, I, 'd, like, a..."
4411547,Once Upon a Time,The Price of Gold,"Adventure, Fantasy, Romance",2011,USA,"[There, is, a, town, in, Maine, I, 'd, like, a..."
...,...,...,...,...,...,...
6965710,Once Upon a Time,The Black Fairy,"Adventure, Fantasy, Romance",2017,USA,"[Previously, on, Once, Upon, A, Time, The, Bla..."
6981874,Once Upon a Time,The Final Battle: Part 1,"Adventure, Fantasy, Romance",2017,USA,"[Previously, on, Once, Upon, a, Time, The, Evi..."
6974322,Once Upon a Time,The Song in Your Heart,"Adventure, Fantasy, Romance",2017,USA,"[Humming, What, 's, this, It, 's, none, of, yo..."
6907086,Once Upon a Time,Tougher Than the Rest,"Adventure, Fantasy, Romance",2017,USA,"[Previously, on, Once, Upon, a, Time, I, wish,..."


In [97]:
ouat_df.to_csv('Once_Upon_a_Time.csv', index=False) 

In [98]:
glee_df = tv_df[tv_df["Series"].isin(["Glee"])]
glee_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3567043,Glee,Acafellas,"Comedy, Drama, Music",2009,USA,"[When, did, you, start, cooking, terri, Oh, it..."
4350999,Glee,Ballad,"Comedy, Drama, Music",2009,USA,"[Here, 's, what, you, missed, last, week, Quin..."
4351000,Glee,Hairography,"Comedy, Drama, Music",2009,USA,"[Here, 's, what, you, missed, last, week, Quin..."
4350997,Glee,Mash-Up,"Comedy, Drama, Music",2009,USA,"[My, weave, What, the, hell, Karofsky, Oh, I, ..."
4351001,Glee,Mattress,"Comedy, Drama, Music",2009,USA,"[Here, 's, what, you, missed, last, week, Emma..."
...,...,...,...,...,...,...
6024329,Glee,The Hurt Locker: Part 2,"Comedy, Drama, Music",2015,USA,"[So, here, 's, what, you, missed, on, Glee, Su..."
6069941,Glee,The Rise and Fall of Sue Sylvester,"Comedy, Drama, Music",2015,USA,"[So, here, 's, what, you, missed, on, Glee, Co..."
6042899,Glee,Transitioning,"Comedy, Drama, Music",2015,USA,"[Shot, through, the, heart, and, you, 're, to,..."
6079027,Glee,We Built This Glee Club,"Comedy, Drama, Music",2015,USA,"[So, here, 's, what, you, missed, on, Glee, Su..."


In [99]:
glee_df.to_csv('Glee.csv', index=False) 

In [100]:
merlin_df = tv_df[tv_df["Series"].isin(["Merlin"])]
merlin_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3359449,Merlin,A Remedy to Cure All Ills,"Adventure, Drama, Fantasy",2008,UK,"[In, a, land, of, myth, and, time, of, magic, ..."
4327443,Merlin,Excalibur,"Adventure, Drama, Fantasy",2008,UK,"[In, a, land, of, myth, and, a, time, of, magi..."
3353791,Merlin,Lancelot,"Adventure, Drama, Fantasy",2008,UK,"[In, a, land, of, myth, and, time, of, magic, ..."
3387000,Merlin,Le Morte d'Arthur,"Adventure, Drama, Fantasy",2008,UK,"[In, a, land, of, myth, and, a, time, of, magi..."
3366586,Merlin,The Beginning of the End,"Adventure, Drama, Fantasy",2008,UK,"[myth, and, the, time, of, magic, the, destiny..."
...,...,...,...,...,...,...
4815579,Merlin,The Disir,"Adventure, Drama, Fantasy",2012,UK,"[In, a, land, of, myth, and, a, time, of, magi..."
4742165,Merlin,The Drawing of the Dark,"Adventure, Drama, Fantasy",2012,UK,"[In, a, land, of, myth, and, a, time, of, magi..."
4725105,Merlin,The Hollow Queen,"Adventure, Drama, Fantasy",2012,UK,"[In, a, land, of, myth, and, a, time, of, magi..."
5026977,Merlin,The Kindness of Strangers,"Adventure, Drama, Fantasy",2012,UK,"[In, a, land, of, myth, and, a, time, of, magi..."


In [101]:
merlin_df.to_csv('Merlin.csv', index=False) 

In [102]:
office_df = tv_df[tv_df["Series"].isin(["The Office"])]
office_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
4284282,The Office,Downsize,"Comedy, Drama",2001,UK,"[I, do, n't, give, shitty, jobs, If, a, good, ..."
4284277,The Office,Judgement,"Comedy, Drama",2001,UK,"[This, is, the, worst, part, of, my, job, you,..."
73428,The Office,New Girl,"Comedy, Drama",2001,UK,"[I, 'm, not, thinking, about, it, I, 'm, doing..."
124692,The Office,The Quiz,"Comedy, Drama",2001,UK,"[VACUUMCLEANERHUMS, I, 'm, 30, today, My, mum,..."
4284281,The Office,Training,"Comedy, Drama",2001,UK,"[What, 's, that, about, Tell, me, Because, at,..."
...,...,...,...,...,...,...
4928290,The Office,Promos,Comedy,2013,USA,"[Oh, oh, She, 's, doing, it, again, Phyllis, h..."
4928292,The Office,Stairmageddon,Comedy,2013,USA,"[What, the, hell, No, this, is, not, happening..."
4777893,The Office,Suit Warehouse,Comedy,2013,USA,"[Stone, Son, Suit, Warehouse, recently, lost, ..."
4867348,The Office,The Farm,Comedy,2013,USA,"[Hold, it, New, glasses, Dwight, what, a, ridi..."


In [103]:
office_df.to_csv('The_Office.csv', index=False) 

In [104]:
parksrec_df = tv_df[tv_df["Series"].isin(["Parks and Recreation"])]
parksrec_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3676950,Parks and Recreation,Beauty Pageant,Comedy,2009,USA,"[Good, morning, everyone, Couple, announcement..."
3555387,Parks and Recreation,Boys' Club,Comedy,2009,USA,"[So, we, 've, been, called, out, to, this, hik..."
4290624,Parks and Recreation,Canvassing,Comedy,2009,USA,"[LESLIE, Well, one, of, the, funner, things, t..."
3615057,Parks and Recreation,Christmas Scandal,Comedy,2009,USA,"[You, are, getting, a, rare, glimpse, At, this..."
3592570,Parks and Recreation,Greg Pikitis,Comedy,2009,USA,"[Greg, Pikitis, You, 're, the, parks, lady, ri..."
...,...,...,...,...,...,...
6003686,Parks and Recreation,Ron & Jammy,Comedy,2015,USA,"[Sighs, Knopey, my, girl, what, up, I, just, f..."
6020598,Parks and Recreation,Save JJ's,Comedy,2015,USA,"[In, front, of, you, are, six, different, wedd..."
6553535,Parks and Recreation,The Johnny Karate Super Awesome Musical Explos...,Comedy,2015,USA,"[It, 's, The, Johnny, Karate, Super, Awesome, ..."
6048227,Parks and Recreation,Two Funerals,Comedy,2015,USA,"[Hey, Hey, guys, listen, Leslie, 's, gon, na, ..."


In [105]:
parksrec_df.to_csv('Parks_and_Recreation.csv', index=False) 

In [52]:
carter_df = tv_df[tv_df["Series"].isin(["Agent Carter"])]
carter_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
6039052,Agent Carter,A Sin to Err,"Action, Adventure, Sci-Fi",2015,USA,"[I, 'm, Peggy, Carter, I, work, for, the, SSR,..."
5993382,Agent Carter,Bridge and Tunnel,"Action, Adventure, Sci-Fi",2015,USA,"[And, now, it, 's, time, for, the, Captain, Am..."
5993377,Agent Carter,Now Is Not the End,"Action, Adventure, Sci-Fi",2015,USA,"[This, thing, 's, moving, too, fast, and, it, ..."
6048131,Agent Carter,SNAFU,"Action, Adventure, Sci-Fi",2015,USA,"[I, 'm, Peggy, Carter, I, work, for, the, SSR,..."
6020508,Agent Carter,The Blitzkrieg Button,"Action, Adventure, Sci-Fi",2015,USA,"[I, 'm, Peggy, Carter, I, work, for, the, SSR,..."
6029695,Agent Carter,The Iron Ceiling,"Action, Adventure, Sci-Fi",2015,USA,"[Previously, on, Agent, Carter, As, of, this, ..."
6002236,Agent Carter,Time and Tide,"Action, Adventure, Sci-Fi",2015,USA,"[I, 'm, Peggy, Carter, During, the, war, I, fo..."
6057673,Agent Carter,Valediction,"Action, Adventure, Sci-Fi",2015,USA,"[Previously, on, Agent, Carter, As, of, this, ..."
6521364,Agent Carter,A Little Song and Dance,"Action, Adventure, Sci-Fi",2016,USA,"[Previously, on, Agent, Carter, Whitney, Frost..."
6469335,Agent Carter,A View in the Dark,"Action, Adventure, Sci-Fi",2016,USA,"[Previously, on, Agent, Carter, This, was, fou..."


In [106]:
carter_df.to_csv('Agent_Carter.csv', index=False) 

In [107]:
daredevil_df = tv_df[tv_df["Series"].isin(["Daredevil"])]
daredevil_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
6112600,Daredevil,Condemned,"Action, Crime, Drama",2015,USA,"[Do, n't, you, move, Do, n't, you, freakin, mo..."
6111677,Daredevil,Cut Man,"Action, Crime, Drama",2015,USA,"[Over, here, Come, on, Help, me, Get, him, out..."
6112359,Daredevil,Daredevil,"Action, Crime, Drama",2015,USA,"[Give, me, a, minute, okay, Mrs, Urich, Um, I,..."
6111959,Daredevil,In the Blood,"Action, Crime, Drama",2015,USA,"[Where, 's, Oleg, It, 's, just, us, now, We, d..."
6111629,Daredevil,Into the Ring,"Action, Crime, Drama",2015,USA,"[Move, please, Matty, Get, out, of, my, way, G..."
6112348,Daredevil,Nelson v. Murdock,"Action, Crime, Drama",2015,USA,"[Oh, God, Would, n't, do, that, if, I, were, y..."
6111760,Daredevil,Rabbit in a Snowstorm,"Action, Crime, Drama",2015,USA,"[Hi, Can, I, get, a, size, 10, and, a, lane, S..."
6112658,Daredevil,Shadows in the Glass,"Action, Crime, Drama",2015,USA,"[Seven, o'clock, a.m, Matt, would, n't, unders..."
6112684,Daredevil,Speak of the Devil,"Action, Crime, Drama",2015,USA,"[Ahhh, Ahh, Ahhh, Started, to, wonder, if, you..."
6112618,Daredevil,Stick,"Action, Crime, Drama",2015,USA,"[Where, is, it, I, do, n't, know, Where, is, t..."


In [108]:
daredevil_df.to_csv('Daredevil.csv', index=False) 

In [109]:
jessicajones_df = tv_df[tv_df["Series"].isin(["Jessica Jones"])]
jessicajones_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
6390779,Jessica Jones,"AKA 1,000 Cuts","Action, Crime, Drama",2015,USA,"[theme_music_playing, grunting, panting, car_s..."
6391195,Jessica Jones,AKA 99 Friends,"Action, Crime, Drama",2015,USA,"[All, it, takes, is, one, word, one, suggestio..."
6390270,Jessica Jones,AKA Crush Syndrome,"Action, Crime, Drama",2015,USA,"[Just, a, few, more, questions, Miss, Jones, I..."
6394383,Jessica Jones,AKA I've Got the Blues,"Action, Crime, Drama",2015,USA,"[It, 's, not, fair, it, 's, mine, Let, him, pl..."
6390424,Jessica Jones,AKA It's Called Whiskey,"Action, Crime, Drama",2015,USA,"[Let, me, down, Um, Robyn, says, you, 're, too..."
6390251,Jessica Jones,AKA Ladies Night,"Action, Crime, Drama",2015,USA,"[New, York, may, be, the, city, that, never, s..."
6393154,Jessica Jones,AKA Sin Bin,"Action, Crime, Drama",2015,USA,"[Jessica, Jessica, Jessica, Smile, Kevin, Be, ..."
6394404,Jessica Jones,AKA Smile,"Action, Crime, Drama",2015,USA,"[Pain, is, always, a, surprise, Help, Somebody..."
6394399,Jessica Jones,AKA Take a Bloody Number,"Action, Crime, Drama",2015,USA,"[I, did, it, I, did, everything, he, said, Wha..."
6391250,Jessica Jones,AKA The Sandwich Saved Me,"Action, Crime, Drama",2015,USA,"[Sync, and, corrections, by, explosiveskull, R..."


In [110]:
jessicajones_df.to_csv('Jessica_Jones.csv', index=False) 

In [111]:
lukecage_df = tv_df[tv_df["Series"].isin(["Luke Cage"])]
lukecage_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
6751120,Luke Cage,Blowin' Up the Spot,"Action, Crime, Drama",2016,USA,"[Luke, stay, with, me, Breathe, Breathe, Just,..."
6749930,Luke Cage,Code of the Streets,"Action, Crime, Drama",2016,USA,"[What, are, you, doing, here, nigga, I, 'm, no..."
6751127,Luke Cage,DWYCK,"Action, Crime, Drama",2016,USA,"[I, should, be, out, there, doing, my, job, Yo..."
6751139,Luke Cage,Episode #1.11,"Action, Crime, Drama",2016,USA,"[We, are, ten, feet, from, the, door, Just, bu..."
6750044,Luke Cage,Just to Get a Rep,"Action, Crime, Drama",2016,USA,"[Bitch, This, ai, n't, no, social, This, is, w..."
6750489,Luke Cage,Manifest,"Action, Crime, Drama",2016,USA,"[Got, that, John, Blaze, shit, Forget, about, ..."
6749923,Luke Cage,Moment of Truth,"Action, Crime, Drama",2016,USA,"[theme_music_playing, Pop, They, lose, 65, gam..."
6751148,Luke Cage,Soliloquy of Chaos,"Action, Crime, Drama",2016,USA,"[All, right, here, 's, the, plan, Triple, cuff..."
6750026,Luke Cage,Step in the Arena,"Action, Crime, Drama",2016,USA,"[All, the, way, Break, it, up, a, little, bit,..."
6750100,Luke Cage,Suckas Need Bodyguards,"Action, Crime, Drama",2016,USA,"[theme_music_playing, Trish, If, you, 're, jus..."


In [112]:
lukecage_df.to_csv('Luke_Cage.csv', index=False) 

In [113]:
ironfist_df = tv_df[tv_df["Series"].isin(["Iron Fist"])]
ironfist_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
6919323,Iron Fist,Bar the Big Boss,"Action, Adventure, Crime",2017,USA,"[Hey, Hello, there, A, little, help, here, No,..."
6919321,Iron Fist,Black Tiger Steals Heart,"Action, Adventure, Crime",2017,USA,"[Sorry, It, 's, all, right, What, Where, are, ..."
6919324,Iron Fist,Dragon Plays with Fire,"Action, Adventure, Crime",2017,USA,"[Now, let, 's, turn, it, over, to, Thembi, Wal..."
6919293,Iron Fist,Eight Diagram Dragon Palm,"Action, Adventure, Crime",2017,USA,"[I, dedicate, myself, to, the, service, of, al..."
6919318,Iron Fist,Felling Tree with Roots,"Action, Adventure, Crime",2017,USA,"[Shit, It, 's, uh, it, 's, Silybum, marianum, ..."
6919317,Iron Fist,Immortal Emerges from Cave,"Action, Adventure, Crime",2017,USA,"[Because, of, who, you, are, there, will, alwa..."
6919322,Iron Fist,Lead Horse Back to Stable,"Action, Adventure, Crime",2017,USA,"[Danny, Davos, You, did, it, Danny, The, trial..."
6919288,Iron Fist,Rolling Thunder Cannon Punch,"Action, Adventure, Crime",2017,USA,"[Wait, Wait, this, is, n't, Stop, We, 're, try..."
6919287,Iron Fist,Shadow Hawk Takes Flight,"Action, Adventure, Crime",2017,USA,"[Corrected, by, GoldenBeard, Ah, There, you, a..."
6919281,Iron Fist,Snow Gives Way,"Action, Adventure, Crime",2017,USA,"[theme_music_playing, Corrected, by, GoldenBea..."


In [114]:
ironfist_df.to_csv('Iron_Fist.csv', index=False) 

In [115]:
defenders_df = tv_df[tv_df["Series"].isin(["The Defenders"])]
defenders_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3860398,The Defenders,Las Vegas v. Reid,"Crime, Drama",2010,USA,"[Man, if, they, ever, catch, this, dirt, bag, ..."
3933881,The Defenders,Las Vegas vs. Johnson,"Crime, Drama",2010,USA,"[The, murder, trial, starts, Wednesday, God, I..."
3871375,The Defenders,Nevada v. Carter,"Crime, Drama",2010,USA,"[Huh, $, 300, Call, Trip, sixes, Have, mercy, ..."
3888164,The Defenders,Nevada v. Cerrato,"Crime, Drama",2010,USA,"[Too, many, tats, Too, pervy, There, he, is, O..."
4052909,The Defenders,Nevada v. Dennis,"Crime, Drama",2010,USA,"[Whoa, whoa, whoa, whoa, whoa, whoa, I, 'm, on..."
3948101,The Defenders,Nevada v. Killa Diz,"Crime, Drama",2010,USA,"[Yeah, I, do, it, like, a, boss, She, tug, at,..."
4087393,The Defenders,Nevada v. Riley,"Crime, Drama",2010,USA,"[Move, move, Get, out, of, the, way, Get, out,..."
3919608,The Defenders,Nevada v. Rodgers,"Crime, Drama",2010,USA,"[He, goes, out, to, collect, a, football, bet,..."
3904320,The Defenders,Nevada v. Sen. Harper,"Crime, Drama",2010,USA,"[I, 'm, sorry, to, just, drop, in, here, witho..."
3846166,The Defenders,Pilot,"Crime, Drama",2010,USA,"[Hey, what, was, that, one, You, 're, ok, It, ..."


In [116]:
defenders_df.to_csv('Defenders.csv', index=False) 

In [117]:
likeme_df = tv_df[tv_df["Series"].isin(["Please Like Me"])]
likeme_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
6785056,Please Like Me,All You Can Eat,"Comedy, Drama, Romance",2013,Australia,"[I, broke, up, with, Geoffrey, What, why, He, ..."
6785000,Please Like Me,French Toast,"Comedy, Drama, Romance",2013,Australia,"[Look, Mae, um, I, 'm, gon, na, have, to, drop..."
5177394,Please Like Me,Horrible Sandwiches,"Comedy, Drama, Romance",2013,Australia,"[Show, 's, over, Peg, Stay, Oh, Jesus, So, how..."
6785033,Please Like Me,Portuguese Custard Tarts,"Comedy, Drama, Romance",2013,Australia,"[Okay, Did, you, have, fun, Did, you, have, a,..."
6784997,Please Like Me,Rhubarb and Custard,"Comedy, Drama, Romance",2013,Australia,"[Josh, Why, ca, n't, i, ever, just, let, mysel..."
6785196,Please Like Me,Spanish Eggs,"Comedy, Drama, Romance",2013,Australia,"[Oh, shit, sorry, mate, Uh, y, you, gon, na, b..."
5813336,Please Like Me,Episode #2.4,"Comedy, Drama, Romance",2014,Australia,"[So, what, did, you, do, today, Oh, my, dance,..."
6785611,Please Like Me,Episode #2.6,"Comedy, Drama, Romance",2014,Australia,"[Why, I, Just, think, I, have, really, bad, mo..."
5798030,Please Like Me,Ham,"Comedy, Drama, Romance",2014,Australia,"[Have, you, spent, time, with, your, mum, whil..."
6785637,Please Like Me,Margherita,"Comedy, Drama, Romance",2014,Australia,"[Still, sad, Still, feeling, sad, Yeah, How, a..."


In [118]:
likeme_df.to_csv('Please_Like_Me.csv', index=False) 

In [119]:
oitnb_df = tv_df[tv_df["Series"].isin(["Orange Is the New Black"])]
oitnb_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
5090872,Orange Is the New Black,Blood Donut,"Comedy, Crime, Drama",2013,USA,"[The, President, does, n't, care, about, the, ..."
5090875,Orange Is the New Black,Bora Bora Bora,"Comedy, Crime, Drama",2013,USA,"[I, 'm, not, gay, All, right, I, 'm, just, say..."
5090878,Orange Is the New Black,Can't Fix Crazy,"Comedy, Crime, Drama",2013,USA,"[Thank, you, so, much, Where, in, the, Nativit..."
5090877,Orange Is the New Black,Fool Me Once,"Comedy, Crime, Drama",2013,USA,"[Did, you, get, in, trouble, too, Sometimes, t..."
5090874,Orange Is the New Black,Fucksgiving,"Comedy, Crime, Drama",2013,USA,"[Why, are, n't, you, at, work, Luschek, never,..."
...,...,...,...,...,...,...
7006757,Orange Is the New Black,Storm-y Weather,"Comedy, Crime, Drama",2017,USA,"[cell_door_slams, theme_song_playing, cell_doo..."
7006791,Orange Is the New Black,Tattoo You,"Comedy, Crime, Drama",2017,USA,"[cell_door_slams, theme_song_playing, cell_doo..."
7007003,Orange Is the New Black,The Reverse Midas Touch,"Comedy, Crime, Drama",2017,USA,"[cell_door_slams, theme_song_playing, cell_doo..."
7006780,Orange Is the New Black,The Tightening,"Comedy, Crime, Drama",2017,USA,"[I, know, I, know, Red, I, 'm, no, Macaulay, C..."


In [120]:
oitnb_df.to_csv('Orange_Is_the_New_Black.csv', index=False) 

In [121]:
queer_df = tv_df[tv_df["Series"].isin(["Queer as Folk"])]
queer_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3118210,Queer as Folk,Episode #1.1,Drama,1999,UK,"[BY, THE, WAY, THAT, 'S, ME, 6, 1, 46-INCH, CH..."
4305574,Queer as Folk,Episode #1.2,Drama,1999,UK,"[Air, Sexy, Boy, Sexyboy, Sexyboy, Sexyboy, Se..."
4305581,Queer as Folk,Episode #1.3,Drama,1999,UK,"[The, thing, about, Alexander, he, 's, a, bit,..."
4305575,Queer as Folk,Episode #1.4,Drama,1999,UK,"[Pulp, Common, People, Singalongwiththe, commo..."
4305576,Queer as Folk,Episode #1.5,Drama,1999,UK,"[@alarm_beeping, @alarm_stops, Right, come, on..."
...,...,...,...,...,...,...
3609852,Queer as Folk,Fuckin' Revenge,"Drama, Romance",2005,USA,"[Poor, Brian, It, was, a, bomb, It, was, a, bo..."
6121817,Queer as Folk,I Love You,"Drama, Romance",2005,USA,"[SONG, One, two, three, four, five, six, take,..."
3795251,Queer as Folk,Mr. Right (Never Broke a Promise),"Drama, Romance",2005,USA,"[SONG, One, two, three, four, five, six, take,..."
3797470,Queer as Folk,We Will Survive!,"Drama, Romance",2005,USA,"[SONG, One, two, three, four, five, six, take,..."


In [122]:
queer_df.to_csv('Queer_as_Folk.csv', index=False) 

In [123]:
lword_df = tv_df[tv_df["Series"].isin(["The L Word"])]
lword_df

,Series,Episode,Genre,Year,Country,Tokenized Texts
3987371,The L Word,"Lookin' at You, Kid","Drama, Romance",2008,Canada,"[Okay, everyone, Quick, round, Table, introduc..."
4296126,The L Word,L'Ennui,"Drama, Romance",2004,"Canada, USA","[Normaal, gesproken, doe, ik, geen, live, scho..."
4296128,The L Word,Lawfully,"Drama, Romance",2004,"Canada, USA","[You, ready, I, 'll, bet, you, are, You, just,..."
5482180,The L Word,Let's Do It,"Drama, Romance",2004,"Canada, USA","[The, L, Word, ep102, Let, 's, Do, It, They, '..."
5622093,The L Word,"Lies, Lies, Lies","Drama, Romance",2004,"Canada, USA","[The, L, Word, ep103, Longing, I, told, you, d..."
...,...,...,...,...,...,...
234905,The L Word,Lifeline,"Drama, Romance",2006,"USA, Canada","[Previously, on, The, L, Word, Where, the, fuc..."
234904,The L Word,Lifesize,"Drama, Romance",2006,"USA, Canada","[This, is, my, cell, Call, me, anytime, Five, ..."
3259629,The L Word,Liquid Heat,"Drama, Romance",2008,"USA, Canada","[It, 's, a, scorcher, out, there, folks, Accor..."
3281023,The L Word,Loyal and True,"Drama, Romance",2008,"USA, Canada","[So, I, might, think, it, 's, a, start, on, ou..."


In [124]:
lword_df.to_csv('The_L_Word.csv', index=False) 